## 中文分词
数据预处理<br>
1.全角变半角<br>
2.将数字变成0，字母变成X，去除空格<br>

In [1]:
import re
# 数据
data = ['他  说  ，  政府  面临  的  首要  任务  是  修改  税制  。', 
        '这  项  呼吁  是  在  ６月３０日  开幕  的  “  通古斯  问题  ９０周年  ”  国际  研讨会  上  发出  的  。',
        '１６日  ２０时３０分  成功  地  定点  于  东经ang８７．５度  赤道  上空  。']
data_origin = [''.join(x.split()) for x in data]
data, data_origin

(['他  说  ，  政府  面临  的  首要  任务  是  修改  税制  。',
  '这  项  呼吁  是  在  ６月３０日  开幕  的  “  通古斯  问题  ９０周年  ”  国际  研讨会  上  发出  的  。',
  '１６日  ２０时３０分  成功  地  定点  于  东经ang８７．５度  赤道  上空  。'],
 ['他说，政府面临的首要任务是修改税制。',
  '这项呼吁是在６月３０日开幕的“通古斯问题９０周年”国际研讨会上发出的。',
  '１６日２０时３０分成功地定点于东经ang８７．５度赤道上空。'])

In [2]:
# 全角变半角
def tohalfwidth(data):
    new_data = list()
    for sentence in data:
        if sentence == '':
            continue
        new_sent = list()
        words = sentence.split()
        for word in words:
            half_word = ''
            for uchar in word:
                inside_code = ord(uchar)
                if inside_code == 12288:
                    inside_code = 32
                elif 65281 <= inside_code <= 65374:
                    inside_code -= 65248
                half_word += chr(inside_code)
            new_sent.append(half_word)
        new_data.append(new_sent)
    return new_data


data = tohalfwidth(data)
data

[['他', '说', ',', '政府', '面临', '的', '首要', '任务', '是', '修改', '税制', '。'],
 ['这',
  '项',
  '呼吁',
  '是',
  '在',
  '6月30日',
  '开幕',
  '的',
  '“',
  '通古斯',
  '问题',
  '90周年',
  '”',
  '国际',
  '研讨会',
  '上',
  '发出',
  '的',
  '。'],
 ['16日', '20时30分', '成功', '地', '定点', '于', '东经ang87.5度', '赤道', '上空', '。']]

In [3]:
# 将数字变成0，字母变成X，去除空格
def generalize(data):
    rNUM = re.compile(r'(-|\+)?(\d+)([\.|·/∶:]\d+)?%?')
    rENG = re.compile(r'[A-Za-z_.]+')
    #rRomaGreece = re.compile(r'[\u2160-\u216f|\u0370-\u03FF]+')
    new_data = list()
    for sent in data:
        new_sent = list()
        for word in sent:
            word = re.sub(r'\s+', '', word)
            word = re.sub(rNUM, '0', word)
            word = re.sub(rENG, 'X', word)
            #word = re.sub(rRomaGreece, 'X', word)
            #word = re.sub(rCon, '-', word)
            new_sent.append(word)
        new_data.append(new_sent)
    return new_data

data = generalize(data)
data

[['他', '说', ',', '政府', '面临', '的', '首要', '任务', '是', '修改', '税制', '。'],
 ['这',
  '项',
  '呼吁',
  '是',
  '在',
  '0月0日',
  '开幕',
  '的',
  '“',
  '通古斯',
  '问题',
  '0周年',
  '”',
  '国际',
  '研讨会',
  '上',
  '发出',
  '的',
  '。'],
 ['0日', '0时0分', '成功', '地', '定点', '于', '东经X0度', '赤道', '上空', '。']]

In [4]:
def postprocess(data_origin, data_seg):

    # data_origin转半角
    data_half = list()
    for sentence in data_origin:
        newsent = list()
        for uchar in sentence:
            inside_code = ord(uchar)
            if inside_code == 12288:
                inside_code = 32
            elif inside_code >= 65281 and inside_code <= 65374:
                inside_code -= 65248
            newchar = chr(inside_code)
            newsent.append(newchar)
        data_half.append(''.join(newsent))

    # print(data_half)
    # print(data_seg)
    # 泛化字符恢复
    rNUM = re.compile(r'(-|\+)?(\d+)([\.|·/∶:]\d+)?(%)?')
    rENG = re.compile(r'[A-Za-z_.]+')
    index = 0
    data_general_recover = list()  # 泛化恢复后的分词结果
    for seg_line, origin_line in zip(data_seg, data_half):
        index += 1
        new_word_list = list()
        list_NUM = list()
        list_ENG = list()

        list_NUM_i = 0
        list_ENG_i = 0

        for t in re.findall(rNUM, origin_line):
            list_NUM.append(''.join(t))
        for t in re.findall(rENG, origin_line):
            list_ENG.append(''.join(t))
        if index == 1453:
            print(origin_line, list_NUM, list_ENG)
        word_list = seg_line.strip().split()
        for word in word_list:
            new_word = list()
            for i in range(len(word)):
                c = word[i]
                if c == '0':
                    new_word.append(list_NUM[list_NUM_i])
                    list_NUM_i += 1
                elif c == 'X':
                    new_word.append(list_ENG[list_ENG_i])
                    list_ENG_i += 1
                else:
                    new_word.append(c)
            new_word = ''.join(new_word)
            new_word_list.append(new_word)
        data_general_recover.append(new_word_list)

    # 切分原始的全角文件
    data_result = list()
    for seg_list, line_origin in zip(data_general_recover, data_origin):
        new_line = []
        len_list = []
        for item in seg_list:
            len_list.append(len(item))

        line_origin = line_origin.strip()
        i = 0
        for w_len in len_list:
            new_line.append(line_origin[i:i + w_len])
            i += w_len
        data_result.append(new_line)
    
    return data_result

data_seg = [' '.join(x) for x in data]
data_result = postprocess(data_origin, data_seg)
data_result

[['他', '说', '，', '政府', '面临', '的', '首要', '任务', '是', '修改', '税制', '。'],
 ['这',
  '项',
  '呼吁',
  '是',
  '在',
  '６月３０日',
  '开幕',
  '的',
  '“',
  '通古斯',
  '问题',
  '９０周年',
  '”',
  '国际',
  '研讨会',
  '上',
  '发出',
  '的',
  '。'],
 ['１６日', '２０时３０分', '成功', '地', '定点', '于', '东经ang８７．５度', '赤道', '上空', '。']]

In [5]:
data = ['他  说  ，  政府  面临  的  首要  任务  是  修改  税制  。', 
        '这  项  呼吁  是  在  ６月３０日  开幕  的  “  通古斯  问题  ９０周年  ”  国际  研讨会  上  发出  的  。',
        '１６日  ２０时３０分  成功  地  定点  于  东经ang８７．５度  赤道  上空  。']
data = generalize(tohalfwidth(data))
data = [''.join(x) for x in data]
# print(data)

def split_single_data(data):
    sents = list()
    labels = list()
    for sent in data:
        Left = 0
        sent = list(sent)
        # print(sent)
        # exit()
        for idx,c in enumerate(sent):
            # if c not in config['special_token']:
            if len(re.sub('\W','',c,flags=re.U))==0:
                if idx > Left:
                    sents.append(list(''.join(sent[Left:idx])))
                    labels.append([0]*len(list(''.join(sent[Left:idx]))))
                    sents.append(c)
                else:
                    sents.append(c)
                Left = idx+1
        if Left!=len(sent):
            sents.append(list(''.join(sent[Left:])))
            labels.append([0]*len(list(''.join(sent[Left:]))))
        sents.append('\n')
    return sents, labels

split_single_data(data)

([['他', '说'],
  ',',
  ['政', '府', '面', '临', '的', '首', '要', '任', '务', '是', '修', '改', '税', '制'],
  '。',
  '\n',
  ['这', '项', '呼', '吁', '是', '在', '0', '月', '0', '日', '开', '幕', '的'],
  '“',
  ['通', '古', '斯', '问', '题', '0', '周', '年'],
  '”',
  ['国', '际', '研', '讨', '会', '上', '发', '出', '的'],
  '。',
  '\n',
  ['0',
   '日',
   '0',
   '时',
   '0',
   '分',
   '成',
   '功',
   '地',
   '定',
   '点',
   '于',
   '东',
   '经',
   'X',
   '0',
   '度',
   '赤',
   '道',
   '上',
   '空'],
  '。',
  '\n'],
 [[0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [6]:
from transformers import (
    AutoConfig,
    AutoTokenizer,
    set_seed
)

bert_path = 'hfl/chinese-electra-180g-base-discriminator'
tokenizer = AutoTokenizer.from_pretrained(
            bert_path
        )

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [7]:
data = ['他  说  ，  政府  面临  的  首要  任务  是  修改  税制  。', 
        '这  项  呼吁  是  在  ６月３０日  开幕  的  “  通古斯  问题  ９０周年  ”  国际  研讨会  上  发出  的  。',
        '１６日  ２０时３０分  成功  地  定点  于  东经ang８７．５度  赤道  上空  。']
# data = generalize(tohalfwidth(data))
# data = [''.join(x) for x in data]
encoded = tokenizer.encode('2009111')
encoded


[101, 8170, 8452, 8148, 102]

2693
